# Fairness - Adult Income Prediction

## Report


### Dataset
The dataset used is Adult Income dataset from Kaggle ([dataset link](https://www.kaggle.com/datasets/wenruliu/adult-income-dataset)).
This dataset can be used to train a model predicting if a given person earns more or less than 50,000 USD a year.

### 1. Model training
First a model (Random Forest) was trained on the preprocessed dataset. Its metrics were as follows:

|               |   recall | precision |       f1 | accuracy |      auc |
|--------------:|---------:|----------:|---------:|---------:|---------:|
| Random Forest | 0.60 |  0.73 | 0.66 |  0.85 | 0.90 |

### 2. Protected attribute selection and statistic
Secondly, fairness statistics with respect to gender were calculated:
|               |   TPR | ACC |       PPV | FPR |      STP |
|--------------:|---------:|----------:|---------:|---------:|---------:|
| Female | 0.87 |  1.14 | 0.96 |  0.26 | 0.31 |

![Random Forest fairness statistics](images/RF_fairness.png "Random Forest fairness statistics")

The model is clearly biased against women - the model is less likely to classify a person as a high-earner if they are female.

### 3. Train and evaluate another model
Another model I chose is Logistic Regression:
|               |   recall | precision |       f1 | accuracy |      auc |
|--------------:|---------:|----------:|---------:|---------:|---------:|
| Logistic Regression | 0.58 |  0.73 | 0.65 |  0.84 | 0.90 |

Its fairness statistics:

|               |   TPR | ACC |       PPV | FPR |      STP |
|--------------:|---------:|----------:|---------:|---------:|---------:|
| Female | 0.81 |  1.15 | 1.03 |  0.2 | 0.27 |

![Logistic Regression fairness statistics](images/LR_fairness.png "Logistic Regression")

Logistic regression seems to fare quite similarly to the Random Forest. It is biased in similar ways.

### 4. Apply bias mitigation
The bias mitigation technique I chose was "resampling". The idea behind it is to sample selected examples multiple times for training to counter the bias visible in the standard training set.

I once again trained a random forest classifier, this time on a resampled training set:

|               |   recall | precision |       f1 | accuracy |      auc |
|--------------:|---------:|----------:|---------:|---------:|---------:|
| Random Forest w/ Resampling | 0.57 |  0.71 | 0.63 |  0.84 | 0.88 |

Its fairness statistics:

|               |   TPR | ACC |       PPV | FPR |      STP |
|--------------:|---------:|----------:|---------:|---------:|---------:|
| Female | 1.51 |  1.10 | 0.63 |  1.59 | 0.85 |

![Resampled random forest fairness statistics](images/RF_resampling_fairness.png "Resampled random forest fairness")

The fairness coefficients changes once we resample the dataset. We tried making the model more fair, however it is now even more unfair (now three instead of two metrics are outside the $[\epsilon, 1/\epsilon]$ interval).
Now the model seems to favour women instead of men.

### 5. Fairness and quality comparison
#### Performance:
|               |   recall | precision |       f1 | accuracy |      auc |
|--------------:|---------:|----------:|---------:|---------:|---------:|
| Random Forest | 0.60 |  0.73 | 0.66 |  0.85 | 0.90 |
| Logistic Regression | 0.58 |  0.73 | 0.65 |  0.84 | 0.90 |
| Random Forest w/ Resampling | 0.57 |  0.71 | 0.63 |  0.84 | 0.88 |

#### Fairness (numbers):
|               |   TPR | ACC |       PPV | FPR |      STP |
|--------------:|---------:|----------:|---------:|---------:|---------:|
| Female (RF) | 0.87 |  1.14 | 0.96 |  0.26 | 0.31 |
| Female (LR) | 0.81 |  1.15 | 1.03 |  0.2 | 0.27 |
| Female (RFw/R) | 1.51 |  1.10 | 0.63 |  1.59 | 0.85 |

#### Fairness (plots):
![Fairness comparison plots](images/all_models_fairness.png "Fairness comparison plots")

Performance deteriorated slightly after resampling. This decrease could be acceptable in a setting in which bias should be mitigated.

### 6. Final Remarks
All the plots above are commented. Appendix contains the code used to produce the results.

## Appendix

### Install dependencies

In [ ]:
!pip install -U pip 'dalex==1.5.0' \
    'shap==0.41.0' 'nbformat>=4.2.0' 'scikit-learn==1.0.2' \
    'pandas==1.3.5' 'numpy==1.22.4' 'lime==0.2.0.1' \
    'alibi==0.8.0' gdown seaborn

### Download data

In [ ]:
import gdown
gdown.download(id='1DRLLOBzyI3JK-hy_TNjCpm937Pujgy9J', output='income.csv', quiet=False)

### Load & Preprocess data
First the data is loaded and some test set size is selected. To make the analysis reproducible, the seed is set.


In [70]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import dalex as dx
import random
import numpy as np
from sklearn.utils import shuffle

SEED = 997
TEST_SIZE = 0.1

data = shuffle(pd.read_csv('income.csv'), random_state=SEED)
data.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')

Dataset contains many categorical variables. Those are one-hot encoded in preprocessing. Generally models can benefit from one-hot encoding the categorical variables. What's more, we translate and scale the input features to follow the N(0, 1) normal distribution. The data is also split into test and train sets.

In [71]:
from typing import Tuple, Optional
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def to_one_hot(data, column):
    dummies = pd.get_dummies(data[column], drop_first=True)
    for dummy_value in dummies.columns:
        data = pd.concat([data, dummies[dummy_value].rename(f'{column}_{dummy_value}')], axis=1)
    return data.drop(column, axis=1)

def scale_dataframe(df: pd.DataFrame, scaler: Optional[StandardScaler] = None) -> Tuple[pd.DataFrame, StandardScaler]:
    res_df = df.copy()
    if scaler is None:
        scaler = StandardScaler()
        scaler = scaler.fit(res_df[res_df.columns])
    res_df[res_df.columns] = scaler.transform(res_df[res_df.columns])
    return res_df, scaler

data = to_one_hot(data, 'workclass')
data = to_one_hot(data, 'marital-status')
data = to_one_hot(data, 'occupation')
data = to_one_hot(data, 'relationship')
data = to_one_hot(data, 'race')
data = to_one_hot(data, 'gender')
data = data.drop(axis=1, columns=['education', 'native-country'])
data

train, test = train_test_split(data, test_size=TEST_SIZE, random_state=SEED)

X_train, train_scaler = scale_dataframe(train.drop(axis = 1, columns=['income']))
y_train = (train['income'] == '>50K').astype(int)

X_test, _ = scale_dataframe(test.drop(axis=1, columns=['income']), train_scaler)
y_test = (test['income'] == '>50K').astype(int)

### 1. Train a model
The basic model that we use is Random Forest Classifier

In [84]:
import dalex as dx
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=SEED)

model.fit(X_train, y_train)

tree_model = model

### 2. Calculate fairness coefficients (protected variable = gender)

In [78]:
protected_variable = X_test.gender_Male.apply(lambda x: "male" if x > 0 else "female")
privileged_group = "male"


In [85]:
def pf(model, df):
    return model.predict_proba(df)[:, 1]

explainer = dx.Explainer(model, X_test, y_test, predict_function=pf, 
    label='Random Forest')
fobject = explainer.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)
explainer.model_performance()
fobject.fairness_check()
fobject.plot()

Preparation of a new explainer is initiated

  -> data              : 4885 rows 44 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 4885 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Random Forest
  -> predict function  : <function pf at 0x7fcfea5680d0> will be used
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.0, mean = 0.24, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)


/home/maciej/Documents/Studia/Zima 2022/XAI/MyRepo/eXplainableMachineLearning-2023/Homeworks/HW5/MaciejPióro/venv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RandomForestClassifier was fitted with feature names



  -> residuals         : min = -1.0, mean = 0.00406, max = 1.0
  -> model_info        : package sklearn

A new explainer has been created!
Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
             TPR       ACC       PPV       FPR       STP
female  0.866232  1.142151  0.961749  0.264706  0.314176


### 3. Train another model - logistic regression

In [86]:
from sklearn.linear_model import LogisticRegression

def pf(model, df):
    return model.predict_proba(df)[:, 1]

model = LogisticRegression(random_state=SEED)

model.fit(X_train, y_train)

explainer_lr = dx.Explainer(model, X_test, y_test, predict_function=pf, label='Logistic regression')
fobject_lr = explainer_lr.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)
fobject_lr.fairness_check()
fobject_lr.plot()

Preparation of a new explainer is initiated

  -> data              : 4885 rows 44 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 4885 values
  -> model_class       : sklearn.linear_model._logistic.LogisticRegression (default)
  -> label             : Logistic regression
  -> predict function  : <function pf at 0x7fcfea4e2af0> will be used
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 1.09e-05, mean = 0.239, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.977, mean = 0.00507, max = 0.995
  -> model_info        : package sklearn

A new explainer has been created!
Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male'. 

/home/maciej/Documents/Studia/Zima 2022/XAI/MyRepo/eXplainableMachineLearning-2023/Homeworks/HW5/MaciejPióro/venv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but LogisticRegression was fitted with feature names



### 4. Apply bias mitigation

In [87]:
from dalex.fairness import resample

protected_variable_train = X_train.gender_Male.apply(lambda x: "male" if x > 0 else "female")
indices_resample = resample(
    protected_variable_train, 
    y_train, 
    type='preferential', # uniform
    probs=tree_model.predict_proba(X_train)[:, 1], # requires probabilities
    verbose=False
)

def pf(model, df):
    return model.predict_proba(df)[:, 1]

model_resample = RandomForestClassifier(random_state=SEED)

model_resample.fit(X_train.iloc[indices_resample, :], y_train.iloc[indices_resample])

explainer_resample = dx.Explainer(model_resample, X_test, y_test, predict_function=pf, label='Random Forest with Resampling')
explainer_resample.model_performance()
fobject_bm = explainer_resample.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)
fobject_bm.fairness_check()
fobject_bm.plot()

Preparation of a new explainer is initiated

  -> data              : 4885 rows 44 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 4885 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Random Forest with Resampling
  -> predict function  : <function pf at 0x7fcfea5070d0> will be used
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.0, mean = 0.222, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -1.0, mean = 0.0213, max = 1.0
  -> model_info        : package sklearn

A new explainer has been created!
Bias detected in 3 metrics: TPR, PPV, FPR

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'male

/home/maciej/Documents/Studia/Zima 2022/XAI/MyRepo/eXplainableMachineLearning-2023/Homeworks/HW5/MaciejPióro/venv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but RandomForestClassifier was fitted with feature names



In [91]:
fobject_bm.plot([fobject_lr, fobject])

In [90]:
pd.concat([explainer.model_performance().result, explainer_lr.model_performance().result, explainer_resample.model_performance().result], axis=0)

,recall,precision,f1,accuracy,auc
Random Forest,0.600336,0.728106,0.658076,0.847902,0.897624
Logistic regression,0.583543,0.726228,0.647114,0.844831,0.902129
Random Forest with Resampling,0.570109,0.709509,0.632216,0.838280,0.875474
